# loading two independent variables

In [5]:
import numpy as np
from qiskit_finance.circuit.library import LogNormalDistribution
# number of qubits per dimension to represent the uncertainty
num_uncertainty_qubits = 3

# parameters for considered random distribution
S = 2.0  # initial spot price
vol = 0.4  # volatility of 40%
r = 0.04  # annual interest rate of 4%
T = 40 / 365  # 40 days to maturity

# resulting parameters for log-normal distribution
mu = (r - 0.5 * vol**2) * T + np.log(S)
sigma = vol * np.sqrt(T)
mean = np.exp(mu + sigma**2 / 2)
variance = (np.exp(sigma**2) - 1) * np.exp(2 * mu + sigma**2)
stddev = np.sqrt(variance)
print(mu)

# lowest and highest value considered for the spot price; in between, an equidistant discretization is considered.
low = np.maximum(0, mean - 3 * stddev)
high = mean + 3 * stddev

# map to higher dimensional distribution
# for simplicity assuming dimensions are independent and identically distributed)
dimension = 2
num_qubits = [num_uncertainty_qubits] * dimension
low = low * np.ones(dimension)
high = high * np.ones(dimension)
mu = mu * np.ones(dimension)
cov = sigma**2 * np.eye(dimension) # covariance matrix

# construct circuit
u = LogNormalDistribution(num_qubits=num_qubits, mu=mu, sigma=cov, bounds=list(zip(low, high)))
# u.measure_all()
u.draw()

0.6887636189161097


┌───────┐
q_0: ┤0      ├
     │       │
q_1: ┤1      ├
     │       │
q_2: ┤2      ├
     │  P(X) │
q_3: ┤3      ├
     │       │
q_4: ┤4      ├
     │       │
q_5: ┤5      ├
     └───────┘

In [14]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, Aer

first_variable_register = QuantumRegister(num_uncertainty_qubits, name='x')
second_variable_register = QuantumRegister(num_uncertainty_qubits, name='y')
second_ancilla_register = QuantumRegister(num_uncertainty_qubits, name='a')
first_classical_register = ClassicalRegister(num_uncertainty_qubits, name='c1')
second_classical_register = ClassicalRegister(num_uncertainty_qubits, name='c2')
third_classical_register = ClassicalRegister(num_uncertainty_qubits, name='c3')

circ = QuantumCircuit(first_variable_register, second_variable_register, second_ancilla_register, first_classical_register, second_classical_register, third_classical_register)
circ.append(u, first_variable_register[:] + second_variable_register[:])
circ.cnot(second_variable_register, second_ancilla_register)
circ.barrier()
# circ.measure(first_variable_register, first_classical_register)
circ.measure(second_variable_register, second_classical_register)
circ.measure(second_ancilla_register, third_classical_register)
circ.draw()

┌───────┐                ░                   
 x_0: ┤0      ├────────────────░───────────────────
      │       │                ░                   
 x_1: ┤1      ├────────────────░───────────────────
      │       │                ░                   
 x_2: ┤2      ├────────────────░───────────────────
      │  P(X) │                ░ ┌─┐               
 y_0: ┤3      ├──■─────────────░─┤M├───────────────
      │       │  │             ░ └╥┘┌─┐            
 y_1: ┤4      ├──┼────■────────░──╫─┤M├────────────
      │       │  │    │        ░  ║ └╥┘┌─┐         
 y_2: ┤5      ├──┼────┼────■───░──╫──╫─┤M├─────────
      └───────┘┌─┴─┐  │    │   ░  ║  ║ └╥┘┌─┐      
 a_0: ─────────┤ X ├──┼────┼───░──╫──╫──╫─┤M├──────
               └───┘┌─┴─┐  │   ░  ║  ║  ║ └╥┘┌─┐   
 a_1: ──────────────┤ X ├──┼───░──╫──╫──╫──╫─┤M├───
                    └───┘┌─┴─┐ ░  ║  ║  ║  ║ └╥┘┌─┐
 a_2: ───────────────────┤ X ├─░──╫──╫──╫──╫──╫─┤M├
                         └───┘ ░  ║  ║  ║  ║  ║ └╥┘
c1: 3/════════════════════════════╬══╬══╬══╬══╬══╬═
                                  ║  ║  ║  ║  ║  ║ 
c2: 3/════════════════════════════╩══╩══╩══╬══╬══╬═
                                  0  1  2  ║  ║  ║ 
c3: 3/═════════════════════════════════════╩══╩══╩═
                                           0  1  2

In [15]:
job = execute(circ, Aer.get_backend('qasm_simulator'), shots=1000)
counts = job.result().get_counts()
print(counts)

{'111 111 000': 10, '100 100 000': 282, '101 101 000': 129, '110 110 000': 44, '001 001 000': 24, '010 010 000': 176, '011 011 000': 335}


# Building Comparator

In [40]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, Aer
from qiskit.quantum_info.operators import Operator

def rotation(k):
    return np.array([[1,0], [0, np.exp(2 * np.pi * 1j / 2**k)]])

def controlled_inverse_rotation_gate(k):
    inverse_rotation = np.array([[1,0], [0, np.exp(-2 * np.pi * 1j / 2**k)]])
    inverse_rotation_gate = Operator(inverse_rotation)
    circuit = QuantumCircuit(1)
    circuit.append(inverse_rotation_gate, [0])
    inverse_rotation_gate = circuit.to_gate(label="R'{}".format(k))
    controlled_rotation = inverse_rotation_gate.control(1)
    return controlled_rotation
    
    
def build_NMSub(m, n):
    first_number_register = QuantumRegister(n+1, name='phi(a)')
    second_number_register = QuantumRegister(m, name='b')
    quantum_circuit = QuantumCircuit(first_number_register, second_number_register)
    counter = 1
    for i in reversed(range(1, n+1)):
        for j in range(counter):
            print(i, j)
            quantum_circuit.append(controlled_inverse_rotation_gate(j+1), [second_number_register[m-j-1]]+ [first_number_register[i]])
        if counter < m:
            counter += 1
        quantum_circuit.barrier()
    for j in range(counter):
        quantum_circuit.append(controlled_inverse_rotation_gate(j+2), [second_number_register[m-j-1]]+ [first_number_register[0]])
    return quantum_circuit

quantum_circuit = build_NMSub(3, 3)
quantum_circuit.draw()

3 0
2 0
2 1
1 0
1 1
1 2


░                ░                       ░ ┌─────┐┌─────┐»
phi(a)_0: ────────░────────────────░───────────────────────░─┤ R'2 ├┤ R'3 ├»
                  ░                ░ ┌─────┐┌─────┐┌─────┐ ░ └──┬──┘└──┬──┘»
phi(a)_1: ────────░────────────────░─┤ R'1 ├┤ R'2 ├┤ R'3 ├─░────┼──────┼───»
                  ░ ┌─────┐┌─────┐ ░ └──┬──┘└──┬──┘└──┬──┘ ░    │      │   »
phi(a)_2: ────────░─┤ R'1 ├┤ R'2 ├─░────┼──────┼──────┼────░────┼──────┼───»
          ┌─────┐ ░ └──┬──┘└──┬──┘ ░    │      │      │    ░    │      │   »
phi(a)_3: ┤ R'1 ├─░────┼──────┼────░────┼──────┼──────┼────░────┼──────┼───»
          └──┬──┘ ░    │      │    ░    │      │      │    ░    │      │   »
     b_0: ───┼────░────┼──────┼────░────┼──────┼──────■────░────┼──────┼───»
             │    ░    │      │    ░    │      │           ░    │      │   »
     b_1: ───┼────░────┼──────■────░────┼──────■───────────░────┼──────■───»
             │    ░    │           ░    │                  ░    │          »
     b_2: ───■────░────■───────────░────■──────────────────░────■──────────»
                  ░                ░                       ░               »
«          ┌─────┐
«phi(a)_0: ┤ R'4 ├
«          └──┬──┘
«phi(a)_1: ───┼───
«             │   
«phi(a)_2: ───┼───
«             │   
«phi(a)_3: ───┼───
«             │   
«     b_0: ───■───
«                 
«     b_1: ───────
«                 
«     b_2: ───────
«

In [59]:
from qiskit.circuit.library import QFT
def build_NMSub(m, n):
    first_number_register = QuantumRegister(n+1, name='phi(a)')
    second_number_register = QuantumRegister(m, name='b')
    quantum_circuit = QuantumCircuit(first_number_register, second_number_register)
    counter = 1
    for i in reversed(range(1, n+1)):
        for j in range(counter):
            # print(i, j)
            quantum_circuit.append(controlled_inverse_rotation_gate(j+1), [second_number_register[m-j-1]]+ [first_number_register[i]])
        if counter < m:
            counter += 1
        # quantum_circuit.barrier()
    for j in range(counter):
        quantum_circuit.append(controlled_inverse_rotation_gate(j+2), [second_number_register[m-j-1]]+ [first_number_register[0]])
    return quantum_circuit.to_gate(label="NMSub")

def build_QNMSub(m, n):
    carry_register = QuantumRegister(1, name='c')
    first_number_register = QuantumRegister(n, name='a')
    second_number_register = QuantumRegister(m, name='b')
    quantum_circuit = QuantumCircuit(carry_register, first_number_register, second_number_register)
    
    quantum_circuit.x(first_number_register[0])
    quantum_circuit.ccx(first_number_register[0], second_number_register[0], carry_register[0])
    quantum_circuit.x(first_number_register[0])
    
    quantum_circuit.x(second_number_register[0])
    quantum_circuit.ccx(first_number_register[0], second_number_register[0], carry_register[0])
    quantum_circuit.x(second_number_register[0])
    # quantum_circuit.barrier()
    qft = QFT(n+1)
    nmSub = build_NMSub(m, n)
    inverse_qft = qft.inverse()
    quantum_circuit.append(qft, carry_register[:] + first_number_register[:])
    quantum_circuit.append(nmSub, carry_register[:] + first_number_register[:] + second_number_register[:])
    quantum_circuit.append(inverse_qft, carry_register[:] + first_number_register[:])
    return quantum_circuit.to_instruction(label="QNMSub")

m = 3
n = 3
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, Aer

carry_register = QuantumRegister(1, name='c')
first_number_register = QuantumRegister(n, name='a')
second_number_register = QuantumRegister(m, name='b')
classical_register = ClassicalRegister(n+1, name='classical')
test_circuit = QuantumCircuit(carry_register, first_number_register, second_number_register, classical_register)
qnmSub = build_QNMSub(m, n)

# load numbers
test_circuit.x(first_number_register[1])
test_circuit.x(second_number_register[1])

test_circuit.append(qnmSub, carry_register[:] + first_number_register[:] + second_number_register[:])
# test_circuit.measure(carry_register[:] + first_number_register[:], classical_register[:])
test_circuit.draw()

    

┌─────────┐
          c: ─────┤0        ├
                  │         │
        a_0: ─────┤1        ├
             ┌───┐│         │
        a_1: ┤ X ├┤2        ├
             └───┘│         │
        a_2: ─────┤3 QNMSub ├
                  │         │
        b_0: ─────┤4        ├
             ┌───┐│         │
        b_1: ┤ X ├┤5        ├
             └───┘│         │
        b_2: ─────┤6        ├
                  └─────────┘
classical: 4/════════════════

In [56]:
job = execute(test_circuit, Aer.get_backend('aer_simulator'), shots=1000)
counts = job.result().get_counts()
print(counts)

{'1100': 31, '1000': 41, '0000': 218, '0010': 503, '0100': 207}


In [81]:
def QComp(n,m):
    carry_register = QuantumRegister(1, name='c')
    first_number_register = QuantumRegister(n, name='a')
    second_number_register = QuantumRegister(m, name='b')
    ancilla_register = QuantumRegister(3, name='ancilla')
    circuit = QuantumCircuit(carry_register, first_number_register, second_number_register, ancilla_register)
    qnmsub = build_QNMSub(m, n)
    circuit.append(qnmsub, carry_register[:] + first_number_register[:] + second_number_register[:])
    circuit.cx(carry_register[:], ancilla_register[0], ctrl_state=0)
    circuit.cx(carry_register[:], ancilla_register[1])
    circuit.ccx(carry_register[:], first_number_register[0], ancilla_register[0], ctrl_state="00")
    circuit.ccx(carry_register[:], first_number_register[0], ancilla_register[2], ctrl_state="00")

    return circuit.to_instruction(label="QComp")


carry_register = QuantumRegister(1, name='c')
first_number_register = QuantumRegister(3, name='a')
second_number_register = QuantumRegister(3, name='b')
ancilla_register = QuantumRegister(3, name='ancilla')
test_circuit = QuantumCircuit(carry_register, first_number_register, second_number_register, ancilla_register)
test_circuit.append(QComp(3,3), carry_register[:] + first_number_register[:] + second_number_register[:] + ancilla_register[:])
test_circuit.decompose().draw()
    
        

┌─────────┐                    
        c: ┤0        ├──o────■────o────o──
           │         │  │    │    │    │  
      a_0: ┤1        ├──┼────┼────o────o──
           │         │  │    │    │    │  
      a_1: ┤2        ├──┼────┼────┼────┼──
           │         │  │    │    │    │  
      a_2: ┤3 QNMSub ├──┼────┼────┼────┼──
           │         │  │    │    │    │  
      b_0: ┤4        ├──┼────┼────┼────┼──
           │         │  │    │    │    │  
      b_1: ┤5        ├──┼────┼────┼────┼──
           │         │  │    │    │    │  
      b_2: ┤6        ├──┼────┼────┼────┼──
           └─────────┘┌─┴─┐  │  ┌─┴─┐  │  
ancilla_0: ───────────┤ X ├──┼──┤ X ├──┼──
                      └───┘┌─┴─┐└───┘  │  
ancilla_1: ────────────────┤ X ├───────┼──
                           └───┘     ┌─┴─┐
ancilla_2: ──────────────────────────┤ X ├
                                     └───┘

# 2 stock call on max

In [62]:
# number of qubits to represent the uncertainty for our probability distribution
num_uncertainty_qubits = 3

# parameters for considered random distribution
S = 2.0  # initial spot price
vol = 0.4  # volatility of 40%
r = 0.05  # annual interest rate of 4%
T = 40 / 365  # 40 days to maturity

# resulting parameters for log-normal distribution
mu = (r - 0.5 * vol**2) * T + np.log(S)
sigma = vol * np.sqrt(T)
mean = np.exp(mu + sigma**2 / 2)
variance = (np.exp(sigma**2) - 1) * np.exp(2 * mu + sigma**2)
stddev = np.sqrt(variance)

# lowest and highest value considered for the spot price; in between, an equidistant discretization is considered.
low = np.maximum(0, mean - 3 * stddev)
high = mean + 3 * stddev

# construct A operator for QAE for the payoff function by
# composing the uncertainty model and the objective
# uncertainty_model = LogNormalDistribution(
#     num_uncertainty_qubits, mu=mu, sigma=sigma**2, bounds=(low, high)
# )
uncertainty_model = LogNormalDistribution(
    num_uncertainty_qubits, mu=mu, sigma=sigma**2, bounds=(low, high)
)

In [73]:
from qiskit.circuit.library import LinearAmplitudeFunction
# set the strike price (should be within the low and the high value of the uncertainty)
strike_price = 1.896

# set the approximation scaling for the payoff function
c_approx = 0.125

# setup piecewise linear objective fcuntion
breakpoints = [low, strike_price] 
# low is the lower bound, strike price is where our payoff function starts to increase
slopes = [0, 1]
# can be float or list of floats.
# for list of floats, the floats are the slopes of the individual linear functions

offsets = [0, 0]
# the offsets of each linear function
f_min = 0
# minimum y value
f_max = high - strike_price
# maximum y value

european_call_objective = LinearAmplitudeFunction(
    num_uncertainty_qubits,
    slopes,
    offsets,
    domain=(low, high),
    image=(f_min, f_max),
    breakpoints=breakpoints,
    rescaling_factor=c_approx,
)
call_objective = european_call_objective.to_gate()

controlled_objective = call_objective.control(1)
# construct A operator for QAE for the payoff function by
# composing the uncertainty model and the objective
# num_qubits = european_call_objective.num_qubits
# european_call = QuantumCircuit(num_qubits)
# european_call.append(european_call_objective, range(num_qubits))
# european_call.draw()

# full circuit

In [77]:
carry_register = QuantumRegister(1, name='c')
first_number_register = QuantumRegister(3, name='a')
second_number_register = QuantumRegister(3, name='b')
first_number_replicate_register = QuantumRegister(3, name='b2')
objective_register = QuantumRegister(1, name='objective')
ancilla_register = QuantumRegister(3, name='ancilla')
ancilla_register2 = QuantumRegister(3, name='ancilla2')

circuit = QuantumCircuit(carry_register, first_number_register, second_number_register, first_number_replicate_register, objective_register, ancilla_register, ancilla_register2)
circuit.append(uncertainty_model, first_number_register[:])
circuit.append(uncertainty_model, second_number_register[:])
circuit.cx(first_number_register[:], first_number_replicate_register[:])
qcomp = QComp(3,3)
circuit.append(qcomp, carry_register[:] + first_number_register[:] + second_number_register[:] + ancilla_register[:])
circuit.append(controlled_objective, [ancilla_register[0]]+ first_number_replicate_register[:] + [objective_register[0]] + ancilla_register2[:])
circuit.append(controlled_objective, [ancilla_register[1]]+ second_number_register[:] + [objective_register[0]] + ancilla_register2[:])
circuit.append(controlled_objective, [ancilla_register[2]]+ first_number_replicate_register[:] + [objective_register[0]] + ancilla_register2[:])

circuit.draw()

┌────────┐                  
         c: ────────────────────────┤0       ├──────────────────
            ┌───────┐               │        │                  
       a_0: ┤0      ├──■────────────┤1       ├──────────────────
            │       │  │            │        │                  
       a_1: ┤1 P(X) ├──┼────■───────┤2       ├──────────────────
            │       │  │    │       │        │                  
       a_2: ┤2      ├──┼────┼────■──┤3       ├──────────────────
            ├───────┤  │    │    │  │        │      ┌────┐      
       b_0: ┤0      ├──┼────┼────┼──┤4       ├──────┤0   ├──────
            │       │  │    │    │  │        │      │    │      
       b_1: ┤1 P(X) ├──┼────┼────┼──┤5       ├──────┤1   ├──────
            │       │  │    │    │  │        │      │    │      
       b_2: ┤2      ├──┼────┼────┼──┤6       ├──────┤2   ├──────
            └───────┘┌─┴─┐  │    │  │  QComp │┌────┐│    │┌────┐
      b2_0: ─────────┤ X ├──┼────┼──┤        ├┤0   ├┤    ├┤0   ├
                     └───┘┌─┴─┐  │  │        ││    ││    ││    │
      b2_1: ──────────────┤ X ├──┼──┤        ├┤1   ├┤    ├┤1   ├
                          └───┘┌─┴─┐│        ││    ││    ││    │
      b2_2: ───────────────────┤ X ├┤        ├┤2   ├┤    ├┤2   ├
                               └───┘│        ││    ││    ││    │
 objective: ────────────────────────┤        ├┤3   ├┤3 F ├┤3   ├
                                    │        ││    ││    ││    │
 ancilla_0: ────────────────────────┤7       ├■    ├┤    ├┤    ├
                                    │        ││  F ││    ││  F │
 ancilla_1: ────────────────────────┤8       ├┤    ├■    ├┤    ├
                                    │        ││    ││    ││    │
 ancilla_2: ────────────────────────┤9       ├┤    ├┤    ├■    ├
                                    └────────┘│    ││    ││    │
ancilla2_0: ──────────────────────────────────┤4   ├┤4   ├┤4   ├
                                              │    ││    ││    │
ancilla2_1: ──────────────────────────────────┤5   ├┤5   ├┤5   ├
                                              │    ││    ││    │
ancilla2_2: ──────────────────────────────────┤6   ├┤6   ├┤6   ├
                                              └────┘└────┘└────┘